# Assignment 1

1. Реализовать Блум фильтр с помощью битового массива. Например, вы можете использовать Битовые операции или библиотеку bitarray.

In [1]:
from bitarray import bitarray

In [2]:
import random


class bloom_filter:

    def __init__(self, n, k, N):
        """
        n - number of bits in the array
        k - number of hashes
        N - array of hashes
        """
        self.arr = bitarray(n)
        self.hashes = random.sample(N, k)

    def insert(self, elem):
        indices = [has(elem) for has in self.hashes]
        for ind in indices:
            self.arr[ind] = 1

    def lookup(self, elem):
        indices = [has(elem) for has in self.hashes]
        for ind in indices:
            if not self.arr[ind]:
                return False
        return True


2. Провести численный эксперимент при false postive rate = 0.05, и количестве объектов S = 1 000 000. Убедится, полученные на семинаре оптимальные параметры фильтра позволяют достичь заявленного false positive rate. Посчитать  $\frac {\epsilon - \hat \epsilon} \epsilon$, где $\hat \epsilon$ - ваша экспериментальная оценка false positive rate. В качестве объектов используйте строки.

In [3]:
from math import log, e, sqrt

Генерируем строчки

In [101]:
import random


def rand_str():
    return ''.join(random.choice('abcdefghijklmnopqrstuvwxyz')
                   for _ in range(random.randint(1, 5)))

arr = [rand_str() for i in range(1000000)]

In [5]:
def optimal_hash(s, n):
    return n/s * log(2)

def optimal_fp(s, n):
    return (1/2) * (n/s) * log(2)

def optimal_size(s, fp):
    return 1/(log(2) ** 2) * s * log(1/fp)

def fp_rate(k, s, fp):
    a = (1 - exp^(-(k*s)/n)) ** k
    if fp <= a:
        return True
    return False

Функции hash

In [90]:
def poly_hashpoly_has(s, x=34, p=1433329):  #1194997): 1190011):
    h = 0
    for j in range(len(s)-1, -1, -1):
        h = (h * x + ord(s[j]) + p) % p
    return h 

def double_hash(y):
    def hash_int(x, a=112, b=22, p=1193483):  #1193483):
        return (a*x + b) % p
    return hash_int(poly_hashpoly_has(y))

def hash1(y): 
    return hash(poly_hashpoly_has(y)) % 6235224

def hash2(y):
    return hash(y) % 6235224

In [91]:
fp = 0.05
s = 1000000
n = int(optimal_size(s, fp))
k = int(optimal_hash(s, n))
N = [hash1, poly_hashpoly_has, double_hash, hash2]

In [92]:
n

6235224

Немного изменим Блум фильтр, чтобы можно было отследить fp

In [93]:
import random


class bloom_filter2:

    def __init__(self, n, k, N):
        """
        n - number of bits in the array
        k - number of hashes
        N - array of hashes
        """
        self.arr = bitarray(n)
        self.hashes = random.sample(N, k)
        self.check = {}
        self.false_p = 0
        self.true_n = 0

    def insert(self, elem):
        indices = []
        for has in self.hashes:
            h = has(elem)
            indices.append(h)
            self.check[elem] = h  #.append(elem)
        for ind in indices:
            self.arr[ind] = 1

    def lookup(self, elem):
        indices = [has(elem) for has in self.hashes]
        a = elem not in self.check
        for ind in indices:
            if not self.arr[ind] and a:
                self.true_n += 1
                return False
        if a:
            self.false_p += 1
        return True

Добавляем элементы в Блум фильтр

In [94]:
bl = bloom_filter2(n, k, N)

for i in arr:
    bl.insert(i)

Тестовая выборка

In [95]:
test_sample = [rand_str() for _ in range(1000000)]

In [96]:
for i in test_sample:
    bl.lookup(i)

In [97]:
fp_exp = bl.false_p / (bl.false_p + bl.true_n)

In [98]:
fp_exp

0.03920929359911665

$\frac {\epsilon - \hat \epsilon} \epsilon$

In [99]:
(fp - fp_exp) / fp

0.21581412801766703

Видно, что для наших данных и подобранных параметров 0.05 действительно является верхней оценкой.